# 음악 순위 크롤링 과제 (멜론, 벅스, 지니)

In [32]:
# Jupyter Notebook 설정 : 모든 결과 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#from fake_useragent import UserAgent
#ua = UserAgent()

# 개발자도구(F12 > Network > Headers > User-Agent)
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}

## 멜론차트 100위까지 가져와 엑셀파일로 저장하기

In [34]:
melon_df = pd.DataFrame(columns=["rank", "title", "artist"])
url = "https://www.melon.com/chart/index.htm"
response = requests.get(url, headers=headers)
response.status_code
bs = BeautifulSoup(response.text, 'html.parser')

def build_dataframe(class_name):
    index = len(melon_df)
    tr_tags = bs.findAll("tr", {"class":class_name})
    for tr_tag in tr_tags:
        rank_tags = tr_tag.select("span.rank")
        if (len(rank_tags) > 0):
            title_tags = tr_tag.select("div.wrap_song_info a")
            artist_tags = tr_tag.select("div.wrap_atist li")
            artist_str = ""
            if (len(artist_tags) > 0):
                for artist_tag in artist_tags:
                    artist_str += ", " + artist_tag.text if bool(artist_str) else artist_tag.text
            else:
                artist_str = title_tags[1].text
            melon_df.loc[index] = [rank_tags[0].text, title_tags[0].text, artist_str]
            index += 1

build_dataframe("lst50")
build_dataframe("lst100")
if (len(melon_df) > 0):
    melon_df.to_excel("site_melon.xlsx", index=False, columns=["rank", "title", "artist"])
    melon_df

200

,rank,title,artist
0,1,Good Goodbye,화사 (HWASA)
1,2,타임캡슐,다비치
2,3,Blue Valentine,NMIXX
3,4,ONE MORE TIME,ALLDAY PROJECT
4,5,Golden,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo..."
...,...,...,...
95,96,Hype Boy,NewJeans
96,97,OVERDRIVE,TWS (투어스)
97,98,Santa Tell Me,Ariana Grande
98,99,천국보다 아름다운,임영웅


## 벅스차트 100위까지 가져와 엑셀파일로 저장하기

In [35]:
bugs_df = pd.DataFrame(columns=["rank", "title", "artist"])
url = "https://music.bugs.co.kr/chart/track/realtime/total?wl_ref=M_contents_03_01"
response = requests.get(url, headers=headers)
response.status_code
bs = BeautifulSoup(response.text, 'html.parser')

index = 0
tr_tags = bs.select("table.list.trackList.byChart tr")
for tr_tag in tr_tags:
    rank_tags = tr_tag.select("div.ranking > strong")
    if (len(rank_tags) > 0):
        title_tags = tr_tag.select("p.title > a")
        artist_tags = tr_tag.select("p.artist > a")
        artist_str = artist_tags[0].text
        if (len(artist_tags) > 1):
            temp = artist_tags[1]["onclick"]
            artists_list = temp.split("\\n")
            for i in range(1, len(artists_list)):
                sindex = artists_list[i].find("||")
                artist_str += ", " + artists_list[i][:sindex]
        bugs_df.loc[index] = [rank_tags[0].text, title_tags[0].text, artist_str]
        index += 1

if (len(bugs_df) > 0):
    bugs_df.to_excel("site_bugs.xlsx", index=False, columns=["rank", "title", "artist"])
    bugs_df


200

,rank,title,artist
0,1,Good Goodbye,화사 (HWASA)
1,2,ONE MORE TIME,ALLDAY PROJECT
2,3,SPAGHETTI (feat. j-hope of BTS),"LE SSERAFIM (르세라핌), j-hope"
3,4,Blue Valentine,NMIXX
4,5,Golden,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo..."
...,...,...,...
95,96,나는 아픈 건 딱 질색이니까,i-dle (아이들)
96,97,네모의 꿈,아이유(IU)
97,98,Do It,Stray Kids (스트레이 키즈)
98,99,Snowman,Sia(시아)


## 지니차트 100위까지 가져와 엑셀파일로 저장하기

In [36]:
import time
import datetime as DT
genie_df = pd.DataFrame(columns=["rank", "title", "artist"])
index = 0
for i in range(2):
    url = "https://www.genie.co.kr/chart/top200?ditc=D&ymd={0}&hh=12&rtm=Y&pg={1}".format(DT.datetime.now().strftime("%Y%m%d"), i+1)
    response = requests.get(url, headers=headers)
    response.status_code
    bs = BeautifulSoup(response.text, 'html.parser')
    tr_tags = bs.select("tr.list")
    for tr_tag in tr_tags:
        rank_tags = tr_tag.select("td.number")
        if (len(rank_tags) > 0):
            rank = rank_tags[0].text.split("\n")[0]
            title_tags = tr_tag.select("td.info > a.title")
            title = title_tags[0].text.strip()
            artist_tags = tr_tag.select("td.info > a.artist")
            artist = artist_tags[0].text.strip().replace(" & ", ", ")
        genie_df.loc[index] = [rank, title, artist]
        index += 1
    time.sleep(1)

if (len(genie_df) > 0):
    genie_df.to_excel("site_genie.xlsx", index=False, columns=["rank", "title", "artist"])
    genie_df

200

200

,rank,title,artist
0,1,Good Goodbye,화사 (HWASA)
1,2,타임캡슐,다비치
2,3,Golden,"HUNTR/X, EJAE, Audrey Nuna, REI AMI, KPop Demo..."
3,4,Drowning,WOODZ
4,5,Blue Valentine,NMIXX
...,...,...,...
95,96,하얀 그리움,프로미스나인
96,97,첫눈처럼 너에게 가겠다,에일리 (Ailee)
97,98,I AM,IVE (아이브)
98,99,첫 만남은 계획대로 되지 않아,TWS (투어스)


## 통합 엑셀파일로 저장하기 (멜론, 벅스, 지니)

In [52]:
melon_df['site'] = 'melon'
bugs_df['site'] = 'bugs'
genie_df['site'] = 'genie'
total_df = pd.concat([melon_df, bugs_df, genie_df])
total_df.to_excel("site_total.xlsx", index=False)

,rank,title,artist,site
0,1,Good Goodbye,화사 (HWASA),melon
0,1,Good Goodbye,화사 (HWASA),bugs
0,1,Good Goodbye,화사 (HWASA),genie
1,2,타임캡슐,다비치,melon
1,2,ONE MORE TIME,ALLDAY PROJECT,bugs
1,2,타임캡슐,다비치,genie
2,3,Blue Valentine,NMIXX,melon
2,3,SPAGHETTI (feat. j-hope of BTS),"LE SSERAFIM (르세라핌), j-hope",bugs
2,3,Golden,"HUNTR/X, EJAE, Audrey Nuna, REI AMI, KPop Demo...",genie


# END